In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import re

## Extract each week

In [134]:
#!pip install pyLDAvis
allweek_data = pd.read_csv("../data_files/submission.csv")
allweek_data = pd.DataFrame(data)

In [135]:
allweek_data.columns

Index(['user_id', 'submission_id', 'payload', 'rubric_id', 'rubric_type',
       'next_action', 'request_origin', 'params', 'code_content'],
      dtype='object')

In [5]:
allweek_data['rubric_id'].unique()

array([10, 15, 32, 38, 44, 61, 65, 72])

In [6]:
rubric_id_10 = allweek_data[allweek_data["rubric_id"] == 10].reset_index(drop=True)
rubric_id_15 = allweek_data[allweek_data["rubric_id"] == 15].reset_index(drop=True)
rubric_id_32 = allweek_data[allweek_data["rubric_id"] == 32].reset_index(drop=True)
rubric_id_38 = allweek_data[allweek_data["rubric_id"] == 38].reset_index(drop=True)
rubric_id_44 = allweek_data[allweek_data["rubric_id"] == 44].reset_index(drop=True)
rubric_id_61 = allweek_data[allweek_data["rubric_id"] == 61].reset_index(drop=True)
rubric_id_65 = allweek_data[allweek_data["rubric_id"] == 65].reset_index(drop=True)
rubric_id_72 = allweek_data[allweek_data["rubric_id"] == 72].reset_index(drop=True)
# Print the resulting rows
print("week1",rubric_id_10.shape) #telecom dataset
print("week2",rubric_id_15.shape) #dropped_airflowdb
print("week3",rubric_id_32.shape) #dropped_fewfiles
print("week5",rubric_id_38.shape) #algorand
print("week6",rubric_id_44.shape) #rag
print("week7",rubric_id_61.shape) #amharic finetuning
print("week9",rubric_id_65.shape) #hardhat
print("week10",rubric_id_72.shape) #image generation
#ruburic_id_10.head()

week1 (53, 9)
week2 (40, 9)
week3 (8, 9)
week5 (36, 9)
week6 (35, 9)
week7 (31, 9)
week9 (31, 9)
week10 (32, 9)


In [7]:
week1_python = allweek_data[allweek_data["rubric_id"] == 10].reset_index(drop=True)
# rubric_id_15 = data[data["rubric_id"] == 15].reset_index(drop=True)
# rubric_id_32 = data[data["rubric_id"] == 32].reset_index(drop=True)
week5_algo = allweek_data[allweek_data["rubric_id"] == 38].reset_index(drop=True)
week6_rag = allweek_data[allweek_data["rubric_id"] == 44].reset_index(drop=True)
week7_finetune = allweek_data[allweek_data["rubric_id"] == 61].reset_index(drop=True)
week9_eth = allweek_data[allweek_data["rubric_id"] == 65].reset_index(drop=True)
week10_img_generate = allweek_data[allweek_data["rubric_id"] == 72].reset_index(drop=True)
# Print the resulting rows
print("week1",week1_python.shape) #telecom dataset
# print("week2",rubric_id_15.shape) #dropped_airflowdb
# print("week3",rubric_id_32.shape) #dropped_fewfiles
print("week5",week5_algo.shape) #algorand
print("week6",week6_rag.shape) #rag
print("week7",week7_finetune.shape) #amharic finetuning
print("week9",week9_eth.shape) #hardhat
print("week10",week10_img_generate.shape) #image generation
#ruburic_id_10.head()

week1 (53, 9)
week5 (36, 9)
week6 (35, 9)
week7 (31, 9)
week9 (31, 9)
week10 (32, 9)


In [11]:
# week1_python.to_csv('../data_files/week1_python.csv', index=False)
# week5_algo.to_csv('../data_files/week5_algo.csv', index=False)
# week6_rag.to_csv('../data_files/week6_rag.csv', index=False)
# week7_finetune.to_csv('../data_files/week7_finetune.csv', index=False)
# week9_eth.to_csv('../data_files/week9_eth.csv', index=False)
# week10_img_generate.to_csv('../data_files/week10_img_generate.csv', index=False)

## Removing Uneccessary Files

In [11]:
# Replace np.nan values with an empty string
# week6_rag['code_content'].fillna('', inplace=True)
def drop_column(df):
    # Drop a column
    columns_to_drop = ['submission_id','payload', 'rubric_id', 'rubric_type', 'next_action', 'request_origin', 'params']
    df = df.drop(columns_to_drop, axis=1)
    print("DataFrame after dropping unwanted column")
    return df

In [1]:
def clean_packages(text):
    patterns = [
    r"### BEGIN File: \w+\.txt ###.*?END File: \w+\.txt ###",
    r"### BEGIN File: (\w+/\w+.txt) ###.*?END File: \1 ###",
    r"### BEGIN File: (\w+/\w+.txt) ###.*?END File: (\w+/\w+.txt) ###",
    r"### BEGIN File: ([\w./-]+\.txt) ###.*?END File: ([\w./-]+\.txt) ###",
    r"### BEGIN File: (\w+/\w+.pgerd) ###.*?END File: (\w+/\w+.pgerd) ###",
    r"### BEGIN File: \w+\.lock ###.*?END File: \w+\.lock ###",
    r"### BEGIN File: \w+\.yml ###.*?END File: \w+\.yml ###",
    r"### BEGIN File: \w+\.yaml ###.*?END File: \w+\.yaml ###",
    r"### BEGIN File: \w+\.toml ###.*?END File: \w+\.toml ###",
    r"### BEGIN File: \w+\.log ###.*?END File: \w+\.log ###",
    #r"### BEGIN File: model_schema\/model_schema_design\.pgerd ###.*?END File: model_schema\/model_schema_design\.pgerd ###",
    r"### BEGIN File: package\.json ###.*?END File: package\.json ###",
    r"### BEGIN File: setup\.py ###.*?END File: setup\.py ###",
    r"### BEGIN File: tailwind.config\.js ###.*?END File: tailwind.config\.js ###",
    r"### BEGIN File: tailwind.config\.ts ###.*?END File: tailwind.config\.ts ###",
    r"### BEGIN File: Dockerfile ###.*?END File: Dockerfile ###",
    ]
    cleaned_text = re.sub("|".join(patterns), '', text, flags=re.DOTALL | re.MULTILINE)
    return cleaned_text
#textt = clean_packages(data['code_content'][151])
# week6_rag['code_content'] = week6_rag['code_content'].apply(clean_packages)

def clean_content(text):
    pattern = r"### BEGIN File: README\.md ###.*?END File: README\.md ###"
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
#data['code_content'][182] = clean_content(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
# week6_rag['code_content'] = week6_rag['code_content'].apply(clean_content)

def clean_License(text):
    pattern = r'BEGIN File: LICENSE.*?END File: LICENSE'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = clean_License(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
# week6_rag['code_content'] = week6_rag['code_content'].apply(clean_License)

def remove_comments(text):
    pattern = r'#.*?\n'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = remove_comments(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
# week6_rag['code_content'] = week6_rag['code_content'].apply(remove_comments)

def clean_perged_files(text):
    pattern = r'### BEGIN File:.*?model_schema/model_schema_design\.pgerd.*?### END File'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = clean_perged_files(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
# week6_rag['code_content'] = week6_rag['code_content'].apply(clean_perged_files)

def extract_content(text):
    pattern = r'Content:\s*(.*?)(?=\s*Commit History:)'
    matches = re.findall(pattern, text, flags=re.DOTALL)
    if matches:
        return '\n'.join(matches)
    else:
        return text
# data['code_content'][182] = extract_content(data['code_content'][182])
# Assuming your DataFrame is named 'df' and the column containing the text is named 'text_column'
# week6_rag['code_content'] = week6_rag['code_content'].apply(extract_content)

def clean_installs(text):
    pattern = r"pip install \w+\n"
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL)
    return cleaned_text
# data['code_content'][182] = clean_perged_files(data['code_content'][182])
# Assuming your DataFrame is named 'data' and the column containing the text is named 'code_content'
# week6_rag['code_content'] = week6_rag['code_content'].apply(clean_installs)

def remove_extra_spaces(strings):
    updated_strings = []
    pattern = r"(?<!\S)\s+(?!\S)"

    for string in strings:
        updated_string = re.sub(pattern, " ", string)
        updated_strings.append(updated_string)

    return updated_strings

def remove_extra_newlines(strings):
    updated_strings = []
    pattern = r"\n+"

    for string in strings:
        updated_string = re.sub(pattern, "\n", string.strip())
        updated_strings.append(updated_string)

    return updated_strings

def extract(text):
    pattern = r'Content:\s*(.*?)$'
    matches = re.findall(pattern, text, flags=re.DOTALL)
    if matches:
        return '\n'.join(matches)
    else:
        return text

def remove_emptycontent_rows(df):
    # Iterate over the rows
    for index, row in df.iterrows():
        if row['code_content'].startswith('Repository Structure:'):
            # Drop the row with the specified index
            df = df.drop(index)
    # Reset the index of the DataFrame
    data = df.reset_index(drop=True)
    return data

def checking_for_empty_value(df):
    column_name = 'code_content'

    # Check if the column contains empty strings
    has_empty_values = any(df[column_name] == '')

    if has_empty_values:
        return "The column contains empty string values."
    else:
        return "The column does not contain empty string values."
    

## Extract basic sample form week1

In [373]:
def read_data(file_path):
    data = pd.read_csv(file_path)
    data = pd.DataFrame(data)
    return data
file_path ="../data_files/week7_finetune.csv"
data = read_data(file_path)

In [374]:
data['code_content'].isnull().value_counts()

code_content
False    21
True     10
Name: count, dtype: int64

In [375]:
# drop none rows from code_content column
data.dropna(subset=['code_content'], inplace=True)

In [376]:
data['code_content'].isnull().value_counts()

code_content
False    21
Name: count, dtype: int64

In [377]:
data = drop_column(data)

DataFrame after dropping unwanted column


In [378]:
data['code_content'] = data['code_content'].apply(clean_packages)
data['code_content'] = data['code_content'].apply(clean_content)
data['code_content'] = data['code_content'].apply(clean_License)
#data['code_content'] = data['code_content'].apply(remove_comments)
data['code_content'] = data['code_content'].apply(clean_perged_files)
data['code_content'] = data['code_content'].apply(extract_content)
data['code_content'] = data['code_content'].apply(clean_installs)

In [379]:
data.loc[10:20]

,user_id,code_content
10,617,"import unittest\nimport sys, os\nsys.path.appe..."
12,746,"## #### Import modules\n%import sys, os, json,..."
13,642,"import unittest\nimport sys, os\nsys.path.appe..."
15,659,"## ### Remove:\n\n## * Null values, new_line(""..."
16,603,"## ### Remove:\n\n## * Null values, new_line(""..."
17,705,import json\nimport re\n\n\nclass Util:\n d...
18,682,! pip install transformers bitsandbytes peft t...
19,585,"## ### Remove:\n\n## * Null values, new_line(""..."
20,763,"## ### Remove:\n\n## * Null values, new_line(""..."


In [380]:
# Drop a row from the dataframe
# data = data.drop(8)
data.shape

(21, 2)

In [381]:
#data['code_content'][10] = extract(data['code_content'][10])
data['code_content'] = data['code_content'].apply(extract)

In [382]:
data['code_content'] = remove_extra_spaces(data['code_content'])

In [383]:
data = remove_emptycontent_rows(data)

In [384]:
checking_for_empty_value(data)

'The column does not contain empty string values.'

In [385]:
data.shape

(21, 2)

In [76]:
# import os
# output_directory = '../content/'
# os.makedirs(output_directory, exist_ok=True)

# # Iterate over each row in the DataFrame
# for index, row in data.iterrows():
#     # Get the code content and create the file path
#     code_content = row['code_content']
#     file_path = os.path.join(output_directory, f'file_{index}.txt')

#     # Write the code content to a text file
#     with open(file_path, 'w') as file:
#         file.write(code_content)

In [386]:
user_data = pd.read_csv("../data_files/email_id_files.csv")
user_data = pd.DataFrame(user_data)
user_data.head()

,Unnamed: 0,Email,User ID,label
0,0,abrhamaddis32@gmail.com,619,Intermediate
1,1,merronmuche@gmail.com,766,Basic
2,2,jaraguta03@gmail.com,665,Basic
3,3,abelbekele.addise@gmail.com,606,Advanced
4,4,amitchew@gmail.com,604,Basic


In [387]:
user_data = user_data.drop('Unnamed: 0', axis=1)

In [388]:
user_data.rename(columns={'User ID': 'user_id'}, inplace=True)

In [389]:
# Merge the DataFrames based on matching user_id values
combined_data = pd.merge(data, user_data, on='user_id', how='inner')

In [390]:
combined_data.head()

,user_id,code_content,Email,label
0,619,import json\nimport re\n \nclass Util():\n de...,abrhamaddis32@gmail.com,Intermediate
1,766,import json\nimport re\n \nclass Util:\n def ...,merronmuche@gmail.com,Basic
2,761,! pip install transformers bitsandbytes peft t...,lillian.alehegn@aait.edu.et,Basic
3,765,"## ### Remove:\n\n## * Null values, new_line(""...",mtadesse813@gmail.com,Basic
4,652,! pip install transformers bitsandbytes peft t...,fanuelabebe@gmail.com,Advanced


In [360]:
combined_data.to_csv('../data_files/all_week_with_comment_labeled/all_week7_finetune.csv', index=False)